<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/unified_compiler_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from __future__ import annotations

import argparse
import csv
import sys
from copy import deepcopy
from dataclasses import dataclass, field
from typing import Any, Callable, Dict, List, Optional, Sequence, Tuple


# ──────────────────────────────────────────────────────────────────────────────
# Data models
# ──────────────────────────────────────────────────────────────────────────────

@dataclass
class CompileConfig:
    max_depth: int = 16
    detect_cycles: bool = True
    omega_phases: List[str] = field(default_factory=list)
    on_step: Optional[Callable[[int, str, "RecursiveOntologyCompiler"], None]] = None


@dataclass
class MindTrace:
    t: int
    symbol: str
    note: Optional[str] = None


@dataclass
class Mind:
    phases: List[str]
    alpha: str = ""
    omega_hits: int = 0
    trace: List[MindTrace] = field(default_factory=list)

    def __post_init__(self):
        self.alpha = self.phases[0] if self.phases else ""


@dataclass
class CompileResult:
    status: str
    depth: int
    alpha: str
    omega_hits: int
    phases_completed: List[str]
    changed: bool
    world_trace: List[Tuple[int, str]]


# ──────────────────────────────────────────────────────────────────────────────
# Defaults and helpers
# ──────────────────────────────────────────────────────────────────────────────

def default_rules() -> Dict[str, Dict[str, str]]:
    """
    Minimal default rules that do not interfere with cycle tests on X/Y.
    They only touch A/B/C so that the cycle injection on X/Y remains clean.
    """
    return {
        "seed_progression": {"A": "B"},
        "progress_progression": {"B": "C"},
        # Keep "C" stable by default; compiler completion is phase-driven.
        "stability": {"C": "C"},
    }


def default_mind(phase_symbols: Sequence[str]) -> Mind:
    return Mind(phases=list(phase_symbols))


def default_phases(mind: Mind) -> List[str]:
    return list(mind.phases)


def world_observer(step: int, symbol: str, compiler: "RecursiveOntologyCompiler") -> None:
    print(f"[world] step={step:02d} symbol={symbol}")


# ──────────────────────────────────────────────────────────────────────────────
# Compiler implementations
# ──────────────────────────────────────────────────────────────────────────────

class RecursiveOntologyCompiler:
    def __init__(
        self,
        symbolic_alphabet: Dict[str, str],
        ruleset: Dict[str, Dict[str, str]],
        config: CompileConfig,
        mind: Mind,
    ):
        self.symbolic_alphabet = symbolic_alphabet
        self.ruleset = ruleset
        self.config = config
        self.mind = mind

        self.ontology_stack: List[str] = []
        self._snapshots: List[List[str]] = []

    def compile(self, seed: str) -> CompileResult:
        # If there are no Ω phases configured, we can never gate → overflow by design.
        if not self.config.omega_phases:
            return CompileResult(
                status="Ω-Overflow",
                depth=0,
                alpha=seed,
                omega_hits=self.mind.omega_hits,
                phases_completed=[],
                changed=False,
                world_trace=[],
            )

        current = seed
        depth = 0
        phases_completed: List[str] = []
        world_trace: List[Tuple[int, str]] = []
        visited_states: List[str] = []  # track in-sequence states for cycle detection
        phase_idx = 0

        while depth < self.config.max_depth:
            # Record world state
            self.ontology_stack.append(current)
            world_trace.append((depth, current))

            # Observer
            if self.config.on_step:
                self.config.on_step(depth, current, self)

            # Advance mind phase (phase-driven engine)
            phase_symbol = self.config.omega_phases[min(phase_idx, len(self.config.omega_phases) - 1)]
            self.mind.trace.append(MindTrace(t=depth, symbol=phase_symbol, note=None))
            if phase_symbol not in phases_completed:
                phases_completed.append(phase_symbol)

            # Ω gate handling: snapshot and complete
            if phase_symbol == "Ω":
                self.mind.omega_hits += 1
                # snapshot ontology at Ω
                self._snapshots.append(list(self.ontology_stack))
                return CompileResult(
                    status="Ω-Complete",
                    depth=depth + 1,
                    alpha=seed,
                    omega_hits=self.mind.omega_hits,
                    phases_completed=phases_completed,
                    changed=(depth > 0),
                    world_trace=world_trace,
                )

            # Prepare next phase index for next iteration (clamp at last if > len-1)
            if phase_idx + 1 < len(self.config.omega_phases):
                phase_idx += 1

            # Apply the first matching world rule in insertion order
            next_state = current
            for _rule_name, mapping in self.ruleset.items():
                if isinstance(mapping, dict) and current in mapping:
                    next_state = mapping[current]
                    break  # apply only one rule per step

            # Cycle detection: if we are about to re-enter a previously seen state, flag it
            if self.config.detect_cycles:
                if next_state in visited_states:
                    return CompileResult(
                        status="Cycle-Detected",
                        depth=depth + 1,
                        alpha=seed,
                        omega_hits=self.mind.omega_hits,
                        phases_completed=phases_completed,
                        changed=True,
                        world_trace=world_trace + [(depth + 1, next_state)],
                    )
                visited_states.append(current)

            # Advance world
            current = next_state
            depth += 1

        # If we exhausted steps without hitting Ω, declare overflow
        return CompileResult(
            status="Ω-Overflow",
            depth=depth,
            alpha=seed,
            omega_hits=self.mind.omega_hits,
            phases_completed=phases_completed,
            changed=(depth > 0),
            world_trace=world_trace,
        )

    def rollback_last(self) -> Optional[List[str]]:
        """
        Restore ontology_stack to the last Ω snapshot if available.
        Returns the snapshot restored, or None if no snapshot exists.
        """
        if not self._snapshots:
            return None
        snapshot = self._snapshots.pop()
        # Restore stack to the snapshot state (not longer than current)
        self.ontology_stack = list(snapshot)
        return snapshot

    @staticmethod
    def write_csv(result: CompileResult, path: str) -> None:
        fieldnames = ["step", "symbol"]
        with open(path, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for step, symbol in result.world_trace:
                writer.writerow({"step": step, "symbol": symbol})


class ConsciousnessMetaCompiler:
    @staticmethod
    def write_csv(mind: Mind, path: str) -> None:
        fieldnames = ["t", "symbol", "note"]
        with open(path, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for rec in mind.trace:
                writer.writerow({"t": rec.t, "symbol": rec.symbol, "note": rec.note or ""})


# ──────────────────────────────────────────────────────────────────────────────
# Smoke tests
# ──────────────────────────────────────────────────────────────────────────────

def smoke_tests() -> None:
    # Base rules for tests
    rules = default_rules()

    # Cycle detection in world (no Self reachable)
    rules_cycle = deepcopy(rules)
    rules_cycle["archetypal_library"] = {"X": "Creator", "Y": "Destroyer"}
    rules_cycle["transmutation_table"] = {"Creator": "Y", "Destroyer": "X"}
    mind2 = default_mind(["α", "β"])  # no Ω symbol to prevent accidental coherence
    phases2 = default_phases(mind2)
    cfg2 = CompileConfig(max_depth=32, detect_cycles=True, omega_phases=phases2)
    compiler2 = RecursiveOntologyCompiler({"X": "", "Y": ""}, rules_cycle, cfg2, mind=mind2)
    res2 = compiler2.compile("X")
    assert res2.status == "Cycle-Detected", f"Expected Cycle-Detected, got {res2.status}"

    # Overflow when phases are empty (no Ω gates)
    cfg3 = CompileConfig(max_depth=5, omega_phases=[])
    mind3 = default_mind(["α"])
    compiler3 = RecursiveOntologyCompiler({"A": ""}, rules, cfg3, mind=mind3)
    res3 = compiler3.compile("A")
    assert res3.status == "Ω-Overflow", f"Expected Ω-Overflow, got {res3.status}"

    # Rollback restores to last Ω snapshot
    mind4 = default_mind(["α", "β", "Ω"])
    compiler4 = RecursiveOntologyCompiler(
        {"A": ""},
        rules,
        CompileConfig(max_depth=16, omega_phases=default_phases(mind4)),
        mind=mind4
    )
    res4 = compiler4.compile("A")
    assert res4.status == "Ω-Complete"
    pre_len = len(compiler4.ontology_stack)
    snap = compiler4.rollback_last()
    assert snap is not None
    post_len = len(compiler4.ontology_stack)
    assert post_len <= pre_len, "Rollback should not increase ontology stack length"

    print("Smoke tests: OK")


# ──────────────────────────────────────────────────────────────────────────────
# Main
# ──────────────────────────────────────────────────────────────────────────────

def main(argv: Optional[List[str]] = None) -> int:
    parser = argparse.ArgumentParser(description="Unified Ontology + Consciousness Compiler")
    parser.add_argument("--seed", default="A", help="Seed symbol to start world compilation")
    parser.add_argument("--max_depth", type=int, default=16, help="Maximum world steps")
    parser.add_argument("--verbose", action="store_true", help="Print world and mind step-by-step")
    parser.add_argument("--world_csv", default="", help="Path to write world trace CSV")
    parser.add_argument("--mind_csv", default="", help="Path to write mind trace CSV")
    parser.add_argument("--no_tests", action="store_true", help="Skip smoke tests")
    args = parser.parse_args(argv)

    if not args.no_tests:
        smoke_tests()

    rules = default_rules()
    mind = default_mind(["α", "β", "Ω"])
    phases = default_phases(mind)

    cfg = CompileConfig(
        max_depth=args.max_depth,
        omega_phases=phases,
        on_step=world_observer if args.verbose else None,
    )

    compiler = RecursiveOntologyCompiler(
        symbolic_alphabet={"A": "seed", "B": "", "C": ""},
        ruleset=rules,
        config=cfg,
        mind=mind,
    )

    result = compiler.compile(args.seed)

    print("\n— Run Summary —")
    print(f"Status: {result.status}")
    print(f"Depth: {result.depth}")
    print(f"Alpha: {result.alpha}")
    print(f"Omega hits: {result.omega_hits}")
    print(f"Phases completed: {', '.join(result.phases_completed) if result.phases_completed else '(none)'}")
    print(f"Changed: {result.changed}")
    print(f"Mind Alpha: {mind.alpha}, Mind Ω hits: {mind.omega_hits}")
    if mind.trace:
        last = mind.trace[-1]
        print(f"Mind last: t={last.t}, symbol={last.symbol}, note={last.note or '(none)'}")

    if args.world_csv:
        RecursiveOntologyCompiler.write_csv(result, args.world_csv)
        print(f"World trace → {args.world_csv}")
    if args.mind_csv:
        ConsciousnessMetaCompiler.write_csv(mind, args.mind_csv)
        print(f"Mind trace → {args.mind_csv}")

    return 0


if __name__ == "__main__":
    sys.exit(main())